In [2]:
import polars as pl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import polars as pl
import scipy.stats as stats
import scipy.sparse as sps
import gc
import tqdm
from polimi.utils._custom import save_sparse_csr

In [3]:
articles = pl.read_parquet('/home/ubuntu/dataset/ebnerd_small/articles.parquet')

behaviors_train = pl.read_parquet('/home/ubuntu/dataset/ebnerd_small/train/behaviors.parquet')
history_train = pl.read_parquet('/home/ubuntu/dataset/ebnerd_small/train/history.parquet')

behaviors_val = pl.read_parquet('/home/ubuntu/dataset/ebnerd_small/validation/behaviors.parquet')
history_val = pl.read_parquet('/home/ubuntu/dataset/ebnerd_small/validation/history.parquet')

contrastive_vector_2 = pl.read_parquet('/home/ubuntu/dataset/Ekstra_Bladet_contrastive_vector/contrastive_vector.parquet')
w_2_vec = pl.read_parquet('/home/ubuntu/dataset/Ekstra_Bladet_word2vec/document_vector.parquet')
roberta = pl.read_parquet('/home/ubuntu/dataset/FacebookAI_xlm_roberta_base/xlm_roberta_base.parquet')
google_bert = pl.read_parquet('/home/ubuntu/dataset/google_bert_base_multilingual_cased/bert_base_multilingual_cased.parquet')
distilbert = pl.read_parquet('/home/ubuntu/dataset/distilbert_title_embedding.parquet')
kenneth = pl.read_parquet('/home/ubuntu/dataset/kenneth_embedding.parquet')
emotions = pl.read_parquet('/home/ubuntu/dataset/emotions_embedding.parquet')



In [4]:
articles_mapping = articles.select('article_id').with_row_index().rename({'index': 'article_index'})
articles_mapping

article_index,article_id
u32,i32
0,3001353
1,3003065
2,3012771
3,3023463
4,3032577
…,…
20733,9803492
20734,9803505
20735,9803525


In [5]:
associations = {
    'contrastive_vector' : contrastive_vector_2,
    'document_vector': w_2_vec,
    'google-bert/bert-base-multilingual-cased': google_bert,
    'FacebookAI/xlm-roberta-base': roberta,
    'title_embedding': distilbert,
    'kenneth_title+subtitle': kenneth,
    'emotion_scores': emotions
    
    
}

In [6]:
for k,value in associations.items():
    ICM_dataframe = value.join(articles, on='article_id').select(['article_id',k]).with_columns(
        pl.col(k).apply(lambda lst : list(range(len(lst)))).alias("indici")      
    )\
    .explode([k,'indici'])\
    .rename({'indici': 'feature_id'})\
    .join(articles_mapping, on='article_id')\
    .drop('article_id')

    n_articles = ICM_dataframe.select('article_index').n_unique()
    n_features = ICM_dataframe.select('feature_id').n_unique()

    ICM = sps.csr_matrix((ICM_dataframe[k].to_numpy(), 
                          (ICM_dataframe["article_index"].to_numpy(), ICM_dataframe["feature_id"].to_numpy())),
                        shape = (n_articles, n_features))
    
    save_sparse_csr(path=Path(f'/home/ubuntu/icm/recsys/small/{k}.npz'),array=ICM)


/tmp/ipykernel_285651/2091541021.py:3: DeprecationWarning: `apply` is deprecated. It has been renamed to `map_elements`.
  pl.col(k).apply(lambda lst : list(range(len(lst)))).alias("indici")
/tmp/ipykernel_285651/2091541021.py:2: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  ICM_dataframe = value.join(articles, on='article_id').select(['article_id',k]).with_columns(


File saved at: /home/ubuntu/icm/recsys/small/contrastive_vector.npz


/tmp/ipykernel_285651/2091541021.py:3: DeprecationWarning: `apply` is deprecated. It has been renamed to `map_elements`.
  pl.col(k).apply(lambda lst : list(range(len(lst)))).alias("indici")
/tmp/ipykernel_285651/2091541021.py:2: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  ICM_dataframe = value.join(articles, on='article_id').select(['article_id',k]).with_columns(


File saved at: /home/ubuntu/icm/recsys/small/document_vector.npz


/tmp/ipykernel_285651/2091541021.py:3: DeprecationWarning: `apply` is deprecated. It has been renamed to `map_elements`.
  pl.col(k).apply(lambda lst : list(range(len(lst)))).alias("indici")
/tmp/ipykernel_285651/2091541021.py:2: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  ICM_dataframe = value.join(articles, on='article_id').select(['article_id',k]).with_columns(


...

File saved at: /home/ubuntu/icm/recsys/small/title_embedding.npz


/tmp/ipykernel_285651/2091541021.py:3: DeprecationWarning: `apply` is deprecated. It has been renamed to `map_elements`.
  pl.col(k).apply(lambda lst : list(range(len(lst)))).alias("indici")
/tmp/ipykernel_285651/2091541021.py:2: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  ICM_dataframe = value.join(articles, on='article_id').select(['article_id',k]).with_columns(


File saved at: /home/ubuntu/icm/recsys/small/kenneth_title+subtitle.npz


/tmp/ipykernel_285651/2091541021.py:3: DeprecationWarning: `apply` is deprecated. It has been renamed to `map_elements`.
  pl.col(k).apply(lambda lst : list(range(len(lst)))).alias("indici")
/tmp/ipykernel_285651/2091541021.py:2: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  ICM_dataframe = value.join(articles, on='article_id').select(['article_id',k]).with_columns(


File saved at: /home/ubuntu/icm/recsys/small/emotion_scores.npz
